In [ ]:
%%capture
%%bash

# Setup the enviroment

module load unstable python-dev python

python -m venv ./python-venv
source ./python-venv/bin/activate

pip install psutil

In [ ]:
%%capture
# %%capture -> suppresses the output of the cell

# change the width of the current notebook
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
%%capture

# Setup the enviroment

import os
import os.path
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
import matplotlib.ticker as ticker
import subprocess

plt.rcParams['font.size'] = '16'

gs = globals()

In [ ]:
%%capture

# Submit jobs for the performance monitoring

# You can always call env and check the available environment variables

# CI has already allocated a job for the runner.
# Therefore we take all the relevant info from the already initialized env vars.
ACCOUNT_=!sacct --format=account -j $SLURM_JOBID | tail -n1 | tr -d ' '

# *****************************************************************************
# Strong Scaling
# *****************************************************************************
# --constraint=clx, i.e., Cascade Lake nodes (Intel codename clx)
# --cpus-per-task=2 -> no multithreading, i.e., one process/task every physical core
# --exclusive, i.e., exclusive use of the node
# --mem=0, i.e., use the whole memory of the node
common_strong_scaling_params="--account=${ACCOUNT_//[[:blank:]]/} --partition=$SLURM_JOB_PARTITION --constraint=clx \
                              --cpus-per-task=2 --exclusive --mem=0 --output=%x.out strong_scaling.batch"

job1=!sbatch --time=0-00:30:00 --job-name=sm_strong_scaling_2    --nodes=1  --ntasks-per-node=2  $common_strong_scaling_params | tail -n 1 | grep -o "job.*" | cut -c 5-

# *****************************************************************************
# Caliper
# *****************************************************************************
common_caliper_params="--account=${ACCOUNT_//[[:blank:]]/} --partition=$SLURM_JOB_PARTITION --constraint=clx \
                       --cpus-per-task=2 --exclusive --mem=0 --output=%x.out caliper.batch"

job1c=!sbatch --time=0-00:15:00 --job-name=sm_caliper_2    --nodes=1  --ntasks-per-node=2  $common_caliper_params | tail -n 1 | grep -o "job.*" | cut -c 5-


In [ ]:
print(job1, job1c)